In [3]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

sql = """
    SELECT *
    FROM `crunchbase-startups.startups.join_table`
"""

df = client.query(sql).to_dataframe()

ModuleNotFoundError: No module named 'google'